In [1]:
import numpy as np
import pandas as pd

In [2]:
def gini_impurity(y):
    classes, counts = np.unique(y, return_counts=True)
    impurity = 1 - np.sum((counts / counts.sum()) ** 2)
    return impurity

In [3]:
def split_dataset(X, y, feature_index, threshold):
    left_mask = X[:, feature_index] <= threshold
    right_mask = X[:, feature_index] > threshold
    return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

In [4]:
def best_split(X, y):
    best_feature, best_threshold, best_gain = None, None, -1
    current_impurity = gini_impurity(y)

    n_samples, n_features = X.shape

    for feature in range(n_features):
        thresholds = np.unique(X[:, feature])
        for threshold in thresholds:
            X_left, y_left, X_right, y_right = split_dataset(X, y, feature, threshold)
            if len(y_left) == 0 or len(y_right) == 0:
                continue

            # Weighted gini
            p = len(y_left) / len(y)
            gini = p * gini_impurity(y_left) + (1 - p) * gini_impurity(y_right)
            gain = current_impurity - gini

            if gain > best_gain:
                best_feature, best_threshold, best_gain = feature, threshold, gain

    return best_feature, best_threshold, best_gain

In [5]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self):
        return self.value is not None

In [6]:
class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        num_labels = len(np.unique(y))

        if (depth >= self.max_depth or num_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feature, threshold, gain = best_split(X, y)
        if feature is None:
            return Node(value=self._most_common_label(y))

        X_left, y_left, X_right, y_right = split_dataset(X, y, feature, threshold)
        left_child = self.build_tree(X_left, y_left, depth + 1)
        right_child = self.build_tree(X_right, y_right, depth + 1)
        return Node(feature, threshold, left_child, right_child)

    def fit(self, X, y):
        self.root = self.build_tree(X, y)

    def predict_one(self, x, node):
        if node.is_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self.predict_one(x, node.left)
        return self.predict_one(x, node.right)

    def predict(self, X):
        return np.array([self.predict_one(x, self.root) for x in X])

    def _most_common_label(self, y):
        vals, counts = np.unique(y, return_counts=True)
        return vals[np.argmax(counts)]

In [7]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train custom Decision Tree
tree = DecisionTree(max_depth=3)
tree.fit(X_train, y_train)

# Predictions
y_pred = tree.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 1.0
